In [54]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import tensorflow as tf


physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(
 #   physical_devices[0], True
#)
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [55]:
df = pd.read_csv("../tags_processed_stages/dafre_tags.csv")
num_classes = df['tags_cat4'].nunique()
print(num_classes)

3486


In [56]:
len(df)

463437

In [57]:

class_counts = df['tags_cat4'].value_counts()
df_filtered = df[df['tags_cat4'].isin(class_counts[class_counts >= 2].index)]
train_df, temp_df = train_test_split(df_filtered, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

common_classes = set(train_df['tags_cat4']).intersection(set(val_df['tags_cat4']), set(test_df['tags_cat4']))

train_df = train_df[train_df['tags_cat4'].isin(common_classes)]
val_df = val_df[val_df['tags_cat4'].isin(common_classes)]
test_df = test_df[test_df['tags_cat4'].isin(common_classes)]
print(val_df['tags_cat4'].nunique())

3095


In [58]:
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [59]:
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [60]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='../fullMin256/', 
    x_col='dir', 
    y_col='tags_cat4', 
    target_size=(150, 150), 
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='../fullMin256/',
    x_col='dir',
    y_col='tags_cat4',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='../fullMin256/',
    x_col='dir',
    y_col='tags_cat4',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 366503 validated image filenames belonging to 3095 classes.
Found 46064 validated image filenames belonging to 3095 classes.
Found 46028 validated image filenames belonging to 3095 classes.


In [61]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.4),
    tf.keras.layers.experimental.preprocessing.RandomTranslation(height_factor=0.2, width_factor=0),
    tf.keras.layers.experimental.preprocessing.RandomTranslation(height_factor=0, width_factor=0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2, 0.2),
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.experimental.preprocessing.RandomContrast(0.2),
])
base_model = tf.keras.applications.MobileNetV2(input_shape=(150, 150, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False
model = Sequential()

model.add(data_augmentation)
model.add(base_model)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(3095, activation='softmax'))



#model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalFocalCrossentropy(), metrics=['accuracy'])

In [62]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True, monitor='val_loss', mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=6, verbose=1, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-5 , mode='min', verbose=1)
sgd_optimizer = tf.keras.optimizers.SGD(learning_rate=0.005, momentum=0.9)
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_categorical_accuracy')])
#model.build(150, 150, 3)

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100, 
    verbose=1,
    steps_per_epoch=len(train_generator)/32,
    validation_steps=len(validation_generator)/32,
    callbacks=[checkpoint, early_stopping, reduce_lr])  


tra_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.figure(figsize=(15,10))
plt.subplot(2,2,1)
plt.plot(tra_acc, label = "Training Accuracy")
plt.plot(val_acc, label = "Validation Accuracy")
plt.legend()
plt.title("Training vs Validation Acc")

tra_loss = history.history['loss']
val_loss = history.history['val_loss']
plt.subplot(2,2,2)
plt.plot(tra_loss, label = "Training Loss")
plt.plot(val_loss, label = "Validation Loss")
plt.legend()
plt.title("Training Loss vs Validation Loss")
plt.show()


2024-01-31 16:54:22.425833: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-31 16:54:22.626685: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/100


2024-01-31 16:54:23.394125: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-31 16:54:28.894422: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f7f15b59a90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-31 16:54:28.894467: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Laptop GPU, Compute Capability 8.6
I0000 00:00:1706738068.976392   84354 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


358/357 [==============================] - ETA: 0s - loss: 9.8782 - accuracy: 0.0187 - top_5_categorical_accuracy: 0.0321

/home/leogu/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


357/357 [==============================] - 51s 126ms/step - loss: 9.8782 - accuracy: 0.0187 - top_5_categorical_accuracy: 0.0321 - val_loss: 9.5194 - val_accuracy: 0.0299 - val_top_5_categorical_accuracy: 0.0674 - lr: 0.0050
Epoch 2/100
175/357 [=============>................] - ETA: 21s - loss: 9.4352 - accuracy: 0.0377 - top_5_categorical_accuracy: 0.0839

In [ ]:
for layer in base_model.layers[:75]:
    layer.trainable = False
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_categorical_accuracy')])
#model.summary(show_trainable=True)
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=400, 
    verbose=1,
    steps_per_epoch=len(train_generator)/32,
    validation_steps=len(validation_generator)/32,
    initial_epoch=history.epoch[-1],
    callbacks=[checkpoint, early_stopping, reduce_lr])  

tra_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.figure(figsize=(15,10))
plt.subplot(2,2,1)
plt.plot(tra_acc, label = "Training Accuracy")
plt.plot(val_acc, label = "Validation Accuracy")
plt.legend()
plt.title("Training vs Validation Acc")

tra_loss = history.history['loss']
val_loss = history.history['val_loss']
plt.subplot(2,2,2)
plt.plot(tra_loss, label = "Training Loss")
plt.plot(val_loss, label = "Validation Loss")
plt.legend()
plt.title("Training Loss vs Validation Loss")
plt.show()


In [ ]:
for layer in base_model.layers:
    layer.trainable = True
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_categorical_accuracy')])
#model.summary(show_trainable=True)
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=500, 
    verbose=1,
    steps_per_epoch=len(train_generator)/32,
    validation_steps=len(validation_generator)/32,
    initial_epoch=history.epoch[-1],
    callbacks=[checkpoint, early_stopping, reduce_lr])  

tra_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.figure(figsize=(15,10))
plt.subplot(2,2,1)
plt.plot(tra_acc, label = "Training Accuracy")
plt.plot(val_acc, label = "Validation Accuracy")
plt.legend()
plt.title("Training vs Validation Acc")

tra_loss = history.history['loss']
val_loss = history.history['val_loss']
plt.subplot(2,2,2)
plt.plot(tra_loss, label = "Training Loss")
plt.plot(val_loss, label = "Validation Loss")
plt.legend()
plt.title("Training Loss vs Validation Loss")
plt.show()
